# RDFLib

## Imports

In [1]:
!pip install rdflib
!pip install PyDrive
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, VOID, DCTERMS, FOAF
from os import listdir, path
import json
import re
import collections
import pandas

## Parse graph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
graph = Graph()
graph.parse("NewOntology.ttl", format="turtle")
print(graph.serialize(format='n3'))

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix iip-srm: <http://purl.org/iip-srm.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

iip-srm:Bird_Look_Like_ a owl:ObjectProperty ;
    rdfs:label "Bird_Look_Like "@en ;
    rdfs:domain iip-srm:Bird ;
    rdfs:range iip-srm:Appearance .

iip-srm:Bird_Speak_Like_ a owl:ObjectProperty ;
    rdfs:label "Bird_Speak_Like "@en ;
    rdfs:domain iip-srm:Bird ;
    rdfs:range iip-srm:Voice .

iip-srm:Description a owl:DatatypeProperty ;
    rdfs:label "Description"@en ;
    rdfs:domain iip-srm:Breed ;
    rdfs:range xsd:string .

iip-srm:Is_Along a owl:DatatypeProperty ;
    rdfs:label "Is_Along"@en ;
    rdfs:domain iip-srm:Appearance ;
    rdfs:range xsd:boolean .

iip-srm:Is_Flying a owl:DatatypeProperty ;
    rdfs:label "Is_Flying"@en ;
    rdfs:domain iip-srm:Appearance ;
    rdfs:range xsd:boolean .

iip-srm:Is_Nestling 

## Generate objects URIs

In [6]:
ontologyUri = "http://www.w3.org/2002/07/owl#"

#### Classes

In [7]:
birdClass = URIRef(ontologyUri + "Bird")
breedClass = URIRef(ontologyUri + "Breed")

amepipClass = URIRef(ontologyUri + "Amepip")
amerobClass = URIRef(ontologyUri + "Amerob")
banswaClass = URIRef(ontologyUri + "Banswa")
comgolClass = URIRef(ontologyUri + "Comgol")
commerClass = URIRef(ontologyUri + "Commer")
comravClass = URIRef(ontologyUri + "Comrav")

voiceClass = URIRef(ontologyUri + "Voice")

imageClass = URIRef(ontologyUri + "Appearance")

#### Object and Data properties

In [8]:
nameDataProp = URIRef(ontologyUri + "Name")
descriptionDataProp = URIRef(ontologyUri + "Description")

lookObjProp = URIRef(ontologyUri + "Bird_Look_Like")

isTakeSmtDataProp = URIRef(ontologyUri + "Is_Take_Smt")
isNestlingDataProp = URIRef(ontologyUri + "Is_Nestling")
isFlyingDataProp = URIRef(ontologyUri + "Is_Flying")
isAlongingDataProp = URIRef(ontologyUri + "Is_Along")

speakObjProp = URIRef(ontologyUri + "Bird_Speak_Like")

voicingDataProp = URIRef(ontologyUri + "Voicing")
jitterDataProp = URIRef(ontologyUri + "Jitter")
pitchDataProp = URIRef(ontologyUri + "Pitch")
pulsesDataProp = URIRef(ontologyUri + "Pulses")
shimmerDataProp = URIRef(ontologyUri + "Shimmer")

#### Classes maps

In [9]:
birdIdentity = {
    "Bird": birdClass,
    "Breed": breedClass,
    "Voice": voiceClass,
    "Appearance": imageClass
}

breedCategories = {
    "Amepip": amepipClass,
    "Amerob": amerobClass,
    "Banswa": banswaClass,
    "Comgol": comgolClass,
    "Commer": commerClass,
    "Comrav": comravClass
}

## Methods of filling the graph with data

In [10]:
def addBird(id, breed, voice, image):
    bird =  URIRef(ontologyUri + str(id))

    graph.add((bird, RDF.type, birdClass))
    graph.add((bird, breedClass, Literal(breed)))
    graph.add((bird, voiceClass, Literal(voice)))
    graph.add((bird, imageClass, Literal(image)))


In [11]:
def addBreed(id, beerd, name, description):
    breed = URIRef(ontologyUri + id)

    graph.add((breed, RDF.type, breedCategories[beerd]))
    graph.add((breed, nameDataProp, Literal(name)))
    graph.add((breed, descriptionDataProp, Literal(description)))


## Add data to graph

In [12]:
lookLikeRefs = dict()
breedRefs = dict()
speakLikeRefs = dict()

In [13]:
def addLookLike(file_name, elements_num):

  with open(file_name, 'r', encoding='utf-8') as f:
      data = json.load(f)

      for id in range(elements_num):
        img = URIRef(ontologyUri + str(data[id]))
        lookLikeRefs[id] = img

        graph.add((img, RDF.type, imageClass))
        graph.add((img, isTakeSmtDataProp, Literal(data[id]["isTakeSmt"])))
        graph.add((img, isNestlingDataProp, Literal(data[id]["isNestling"])))
        graph.add((img, isFlyingDataProp, Literal(data[id]["isFlying"])))
        graph.add((img, isAlongingDataProp, Literal(data[id]["isAlonging"])))


In [14]:
def addSpeakLike(file_name, elements_num):
  with open(file_name, 'r', encoding='utf-8') as f:
      data = json.load(f)

      for id in range(elements_num):
        voice = URIRef(ontologyUri + str(data[id]))
        speakLikeRefs[id] = voice

        graph.add((voice, RDF.type, voiceClass))
        graph.add((voice, voicingDataProp, Literal(data[id]["voicing"])))
        graph.add((voice, jitterDataProp, Literal(data[id]["jitter"])))
        graph.add((voice, pitchDataProp, Literal(data[id]["pitch"])))
        graph.add((voice, pulsesDataProp, Literal(data[id]["pulses"])))
        graph.add((voice, shimmerDataProp, Literal(data[id]["shimmer"])))

In [15]:
# Добавляем породы и описание
breedRefs[0] = addBreed("0", "Amepip", "Buff bellied Pipit", "The buff-bellied pipit or American pipit (Anthus rubescens) is a small songbird found on both sides of the northern Pacific. It is known as American pipit in North America and buff-bellied pipit in Eurasia. Like most other pipits, the buff-bellied pipit is an undistinguished-looking species which usually can be seen to run around on the ground. The rubescens subspecies has lightly streaked grey-brown upperparts and is diffusely streaked below on the buff breast and flanks. The belly is whitish, the bill and legs are dark. The japonicus subspecies is darker above and has bolder black streaking on its whiter underparts; its legs have a reddish hue")
breedRefs[1] = addBreed("1", "Amerob", "American Robin", "The American robin (Turdus migratorius) is a migratory bird of the true thrush genus and Turdidae, the wider thrush family. It is named after the European robin because of its reddish-orange breast, though the two species are not closely related, with the European robin belonging to the Old World flycatcher family.  The head varies from jet black to gray, with white eye arcs and white supercilia. The throat is white with black streaks, and the belly and undertail coverts are white. The American robin has a brown back and a reddish-orange breast, varying from a rich red maroon to peachy orange. The bill is mainly yellow with a variably dark tip, the dusky area becoming more extensive in winter, and the legs and feet are brown. The sexes are similar, but females tend to be duller in color than males.")
breedRefs[2] = addBreed("2", "Banswa", "Pale Sand Martin", "The pale martin or pale sand martin (Riparia diluta) is a small passerine bird in the swallow family. It is found in open habitats such as farmland, grassland and savannah, usually near water. It is found from Central Asia to southeastern China. The species was formerly considered a subspecies of the sand martin.")
breedRefs[3] = addBreed("3", "Comgol", "Common Goldeneye", "The common goldeneye or simply goldeneye (Bucephala clangula) is a medium-sized sea duck of the genus Bucephala, the goldeneyes. Its closest relative is the similar Barrow's goldeneye. The common goldeneye has a wingspan of 77-83 cm. The species is named for its golden-yellow eye. Adult males have a dark head with a greenish gloss and a circular white patch below the eye, a dark back and a white neck and belly. Adult females have a brown head and a mostly grey body. Their legs and feet are orange-yellow.")
breedRefs[4] = addBreed("4", "Commer", "Common Merganser", "The common merganser (North American) or goosander (Eurasian) (Mergus merganser) is a large seaduck of rivers and lakes in forested areas.  Like other species in the genus Mergus, it has a crest of longer head feathers, but these usually lie smoothly rounded behind the head, not normally forming an erect crest. Adult males in breeding plumage are easily distinguished, the body white with a variable salmon-pink tinge, the head black with an iridescent green gloss, the rump and tail grey, and the wings largely white on the inner half, black on the outer half.")
breedRefs[5] = addBreed("5", "Comrav", "Common Raven", "The common raven (Corvus corax) is a large all-black passerine bird. It is the most widely distributed of all corvids. Integrated into the nominate subspecies in the Lake Baikal region. It has a distinctly larger and thicker bill than the nominate subspecies does.")

In [ ]:
#Добавляем изображения
addLookLike('META_IMG.json',60)

In [ ]:
#Добавляем звуки
addSpeakLike('amepip.json',5)
addSpeakLike('amerob.json',5)
addSpeakLike('banswa.json',5)
addSpeakLike('comgol.json',5)
addSpeakLike('commer.json',5)
addSpeakLike('comrav.json',5)

In [19]:
#Добавляем птицам свойства по классам
voice = 0

for id in range(5):
  addBird(voice, breedRefs[0], speakLikeRefs[voice], lookLikeRefs[id])
  voice=+1

for id in range(10,14):
  addBird(voice, breedRefs[1], speakLikeRefs[voice], lookLikeRefs[id])
  voice=+1

for id in range(20,24):
  addBird(voice, breedRefs[2], speakLikeRefs[voice], lookLikeRefs[id])
  voice=+1

for id in range(30,34):
  addBird(voice, breedRefs[3], speakLikeRefs[voice], lookLikeRefs[id])
  voice=+1

for id in range(40,44):
  addBird(voice, breedRefs[4], speakLikeRefs[voice], lookLikeRefs[id])
  voice=+1

for id in range(50,54):
  addBird(voice, breedRefs[5], speakLikeRefs[voice], lookLikeRefs[id])

In [48]:
result = graph.query("""
PREFIX :  <http://visualdataweb.org/newOntology/>
SELECT ?Appearance
WHERE {
    :Bird_Look_Like rdfs:label ?Appearance .
}""")

for row in result:
    print("%s" % row)